## Классификация текстов

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import random
random.seed(1228)
from sklearn.feature_extraction.text import *
from sklearn.metrics import *
from sklearn.pipeline import Pipeline
import re
from pymystem3 import Mystem
import numpy as np
import itertools
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from nltk.stem.snowball import RussianStemmer
import seaborn as sns

Разбиение тектов на обучающее и тестовое множествоё

In [3]:
m = Mystem()
regex = re.compile('[А-яа-я]+')

def words_only(text, regex=regex):
    try:
        return ' '.join(regex.findall(text))
    except:
        return ''
    
def lemmatize(text, mystem=m):
    try:
        return ' '.join(m.lemmatize(text)).strip()
    except:
        return ''

def stemming(text, stemmer=RussianStemmer()):
    try:
        return ' '.join([stemmer.stem(w) for w in text.split()])
    except:
        return ''

In [4]:
data = pd.read_csv('datasets/nlp/lenta.csv', usecols=['text', 'class'])

data['class'].value_counts()

Россия               82
Экономика            62
Интернет и СМИ       54
Мир                  53
Спорт                52
Из жизни             46
Бывший СССР          38
Наука и техника      38
Силовые структуры    35
Культура             35
Ценности             24
Путешествия          19
Name: class, dtype: int64

In [5]:
data.head()

,text,class
0,Четырехкратный олимпийский чемпион и кандидат ...,Спорт
1,Трое курганских полицейских предстанут перед с...,Силовые структуры
2,"Дизайнер Николя Гескьер, креативный директор ж...",Ценности
3,Бойцы Вооруженных сил Украины во время испытан...,Бывший СССР
4,Басманный суд Москвы арестовал до 21 июля бывш...,Силовые структуры


In [7]:
stemming('потребитель')

'потребител'

In [10]:
corpus_sample_train = pd.DataFrame(columns=data.columns)
corpus_sample_test = pd.DataFrame(columns=data.columns)

for cl in data['class'].unique():
    corpus_sample = data[data['class'] == cl]
    if len(corpus_sample) > 30:
        corpus_sample_train = corpus_sample_train.append(corpus_sample.iloc[:30, :])
        corpus_sample_test = corpus_sample_test.append(corpus_sample.iloc[30:, :])
    elif len(corpus_sample) > 50:
        corpus_sample_train = corpus_sample_train.append(corpus_sample.iloc[:15, :])
        corpus_sample_test = corpus_sample_test.append(corpus_sample.iloc[15:, :])

In [11]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

clf = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', RandomForestClassifier())
])

clf.fit(corpus_sample_train['text'], corpus_sample_train['class'])

Pipeline(steps=[('vect', CountVectorizer()), ('clf', RandomForestClassifier())])

In [12]:
true = corpus_sample_test['class']
predictions = clf.predict(corpus_sample_test['text'])

In [13]:
predictions

array(['Спорт', 'Спорт', 'Культура', 'Спорт', 'Спорт', 'Спорт', 'Спорт',
       'Спорт', 'Спорт', 'Культура', 'Спорт', 'Спорт', 'Экономика',
       'Спорт', 'Спорт', 'Спорт', 'Спорт', 'Спорт', 'Культура', 'Спорт',
       'Спорт', 'Мир', 'Из жизни', 'Силовые структуры', 'Экономика',
       'Силовые структуры', 'Наука и техника', 'Бывший СССР',
       'Бывший СССР', 'Бывший СССР', 'Бывший СССР', 'Бывший СССР',
       'Бывший СССР', 'Экономика', 'Бывший СССР', 'Россия',
       'Интернет и СМИ', 'Из жизни', 'Россия', 'Экономика',
       'Наука и техника', 'Россия', 'Россия', 'Экономика', 'Россия',
       'Силовые структуры', 'Россия', 'Россия', 'Культура', 'Мир', 'Мир',
       'Культура', 'Из жизни', 'Культура', 'Мир', 'Интернет и СМИ',
       'Культура', 'Россия', 'Спорт', 'Россия', 'Интернет и СМИ',
       'Экономика', 'Культура', 'Россия', 'Россия', 'Россия', 'Россия',
       'Россия', 'Из жизни', 'Экономика', 'Культура', 'Наука и техника',
       'Культура', 'Мир', 'Культура', 'Из жизн